In [ ]:
from pathlib import Path

import numpy as np
import matplotlib.pyplot as plt

from leap3d.scanning import ScanParameters, ScanResults
from leap3d.config import DATA_DIR, PARAMS_FILEPATH, ROUGH_COORDS_FILEPATH, MELTING_POINT
from leap3d.plotting import plot_top_layer_temperature

In [ ]:
scan_params = ScanParameters(PARAMS_FILEPATH, ROUGH_COORDS_FILEPATH, case_index=20)

In [ ]:
case_index = 20
case_filename = DATA_DIR / f"case_{case_index:04}.npz"


In [ ]:
scan_results = ScanResults(case_filename)

In [ ]:
scan_results.get_laser_coordinates_at_timestep(0)

In [ ]:
(x_coords, y_coords, z_coords), interpolated_values = scan_results.get_interpolated_grid_around_laser(2000, 16, 0.25, scan_params, include_high_resolution_points=True)

In [ ]:
(x_coords2, y_coords2, z_coords2), interpolated_values2 = scan_results.get_interpolated_grid_around_laser(2000, 16, 0.25, scan_params, include_high_resolution_points=False)

In [ ]:
assert (x_coords == x_coords2).all()
assert (y_coords == y_coords2).all()
assert (z_coords == z_coords2).all()

In [ ]:
interpolated_values.shape

In [ ]:
laser_x, laser_y = scan_results.get_laser_coordinates_at_timestep(2000)

In [ ]:
fig, ax = plt.subplots()
rough_temps = scan_results.get_rough_temperatures_at_timestep(2000)
ax.set_aspect('equal')

ax.set_xlim(np.min(x_coords), np.max(x_coords))
ax.set_ylim(np.min(y_coords), np.max(y_coords))

plot_top_layer_temperature(ax, rough_temps[:, :, -1], scan_params, vmin=0, vmax=MELTING_POINT)
# ax.pcolormesh(x_coords, y_coords, interpolated_values.reshape((64, 64)))

ax.scatter(laser_x, laser_y, c='r', s=10, marker='x')
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.pcolormesh(x_coords, y_coords, interpolated_values.reshape((64, 64)), vmin=0, vmax=MELTING_POINT)

ax.scatter(laser_x, laser_y, c='r', s=10, marker='x')
fig.show()

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.pcolormesh(x_coords, y_coords, interpolated_values2.reshape((64, 64)), vmin=0, vmax=MELTING_POINT)

ax.scatter(laser_x, laser_y, c='r', s=10, marker='x')
fig.show()

In [ ]:
from scipy import interpolate

scan_parameters = scan_params
timestep = 2000
step_scale = 1 / 4
rough_coordinates = scan_parameters.rough_coordinates
x_rough = rough_coordinates['x_rough']
y_rough = rough_coordinates['y_rough']
z_rough = rough_coordinates['z_rough']
grid_points = ([x[0][0] for x in x_rough], [y[0] for y in y_rough[0]], [z for z in z_rough[0][0]])

rough_temperature = scan_results.get_rough_temperatures_at_timestep(timestep)

points_to_interpolate = scan_parameters.get_coordinates_around_position(laser_x, laser_y, 16,
                                                                scale=step_scale, is_3d=False)

# if include_high_resolution_points:
# points, values = self.get_coordinate_points_and_temperature_at_timestep(scan_parameters, timestep)
# interpolated_values = interpolate.griddata(points, values, points_to_interpolate, method='linear')

# high_res_points = self.
high_res_points, high_res_temps = scan_results.get_melt_pool_coordinates_and_temperature(timestep)

step_size = scan_parameters.rough_coordinates_step_size * step_scale
half_step = step_size / 2

x_min = np.min(x_rough) + half_step
x_max = np.max(x_rough) - half_step
y_min = np.min(y_rough) + half_step
y_max = np.max(y_rough) - half_step
z_min = np.min(z_rough)
z_max = np.max(z_rough)
upscaled_x_range = np.arange(x_min, x_max, step_size)
upscaled_y_range = np.arange(y_min, y_max, step_size)
upscaled_z_range = np.arange(z_min, z_max + step_size, step_size)

upscaled_rough_coordinates = [(x, y, z) for x in upscaled_x_range for y in upscaled_y_range for z in upscaled_z_range]
upscaled_rough_temperatures = interpolate.interpn(grid_points, rough_temperature, upscaled_rough_coordinates, method='linear', bounds_error=False, fill_value=None)

upscaled_rough_temperatures = upscaled_rough_temperatures.reshape(upscaled_x_range.shape[0], upscaled_y_range.shape[0], upscaled_z_range.shape[0])


print(len(high_res_points))
print(upscaled_rough_temperatures.shape)
for high_res_point, high_res_temp in zip(high_res_points, high_res_temps):
    x, y, z = high_res_point
    x_index = np.argmin(np.abs(upscaled_x_range - x))
    y_index = np.argmin(np.abs(upscaled_y_range - y))
    z_index = np.argmin(np.abs(upscaled_z_range - z))
    print(x_index, y_index, z_index)
    print(step_size, np.min(np.abs(upscaled_x_range - x)),
          np.min(np.abs(upscaled_y_range - y)),
          np.min(np.abs(upscaled_z_range - z)))

    print(upscaled_rough_temperatures[x_index, y_index, z_index], high_res_temp)
    upscaled_rough_temperatures[x_index, y_index, z_index] = high_res_temp

print(upscaled_x_range.shape)
print(upscaled_y_range.shape)
print(upscaled_z_range.shape)


In [ ]:
def get_coordinate_ranges(points):
    points = np.array(points)
    x_min = np.min(points[:, 0])
    x_max = np.max(points[:, 0])
    y_min = np.min(points[:, 1])
    y_max = np.max(points[:, 1])
    z_min = np.min(points[:, 2])
    z_max = np.max(points[:, 2])

    return x_min, x_max, y_min, y_max, z_min, z_max

In [ ]:
total_timesteps = scan_results.total_timesteps

x_range = None
y_range = None
z_range = None

for timestep in range(total_timesteps):
    points, _ = scan_results.get_melt_pool_coordinates_and_temperature(timestep)
    if len(points) == 0:
        continue
    x_min, x_max, y_min, y_max, z_min, z_max = get_coordinate_ranges(points)
    new_x_range = x_max - x_min
    x_range = new_x_range if x_range is None else max(x_range, new_x_range)
    new_y_range = y_max - y_min
    y_range = new_y_range if y_range is None else max(y_range, new_y_range)
    new_z_range = z_max - z_min
    z_range = new_z_range if z_range is None else max(z_range, new_z_range)

print(f"x_range: {x_range}")
print(f"y_range: {y_range}")
print(f"z_range: {z_range}")

In [ ]:
rough_coords = scan_params.rough_coordinates
x_rough = rough_coords['x_rough']
y_rough = rough_coords['y_rough']
z_rough = rough_coords['z_rough']

x_rough_min = np.min(x_rough)
x_rough_max = np.max(x_rough)
y_rough_min = np.min(y_rough)
y_rough_max = np.max(y_rough)
z_rough_min = np.min(z_rough)
z_rough_max = np.max(z_rough)

print(f"x_rough_range: {(x_rough_max - x_rough_min) * 1000}")
print(f"y_rough_range: {(y_rough_max - y_rough_min) * 1000}")
print(f"z_rough_range: {(z_rough_max - z_rough_min) * 1000}")

In [ ]:
(x_rough[1][0][0] - x_rough[0][0][0]) * 8


In [ ]:
from scipy import interpolate

rough_coordinates_points_list = scan_params.get_rough_coordinates_points_list()

In [ ]:
timestep = 1500

In [ ]:
def get_interpolated_box_around_laser(rough_coordinates, scan_results, timestep, scale, method='nearest'):
    x_rough = rough_coordinates['x_rough']
    y_rough = rough_coordinates['y_rough']
    z_rough = rough_coordinates['z_rough']
    rough_temps = scan_results.get_rough_temperatures_at_timestep(timestep)
    laser_x, laser_y = scan_results.get_laser_coordinates_at_timestep(timestep)
    # step_size = scan_params.rough_coordinates_step_size
    step_size = x_rough[1][0][0] - x_rough[0][0][0]
    x_start = laser_x - (step_size * (4 - 0.5 * scale))
    x_end = laser_x + (step_size * (4 + 0.5 * scale))
    y_start = laser_y - (step_size * (4 - 0.5 * scale))
    y_end = laser_y + (step_size * (4 + 0.5 * scale))

    x_points = np.arange(x_start, x_end, step_size * scale)
    y_points = np.arange(y_start, y_end, step_size * scale)
    z_points = np.array(z_rough[0][0])
    desired_points = [(x_point, y_point, z_point) for x_point in x_points for y_point in y_points for z_point in z_points]

    grid_points = ([x[0][0] for x in x_rough], [y[0] for y in y_rough[0]], z_points)
    interpolated_temperature = interpolate.interpn(grid_points, rough_temps[:, :, :], desired_points, method=method)

    return x_points, y_points, z_points, interpolated_temperature

In [ ]:
scale = 0.25

In [ ]:
timestep = 3000

In [ ]:
x_points, y_points, z_points, interpolated_temperature = get_interpolated_box_around_laser(rough_coords, scan_results, timestep, 0.25)

In [ ]:
print(x_points.max() - x_points.min())
print(y_points.max() - y_points.min())

In [ ]:
rough_temps = scan_results.get_rough_temperatures_at_timestep(timestep)
laser_x, laser_y = scan_results.get_laser_coordinates_at_timestep(timestep)

In [ ]:
from matplotlib import pyplot as plt

def plot_interolated_top(ax, x_points, y_points, z_points, interpolated_temperature, laser_x, laser_y):
    ax.set_aspect('equal')
    x_image_points = np.array([np.array([x] * len(y_points)) for x in x_points])
    y_image_points = np.array([y_points for _ in x_points])

    im = ax.pcolormesh(x_image_points, y_image_points, interpolated_temperature.reshape((len(x_points), len(y_points), len(z_points)))[:, :, -1], vmin=0, vmax=MELTING_POINT, shading='nearest')
    im = ax.scatter(laser_x, laser_y, c='red')


In [ ]:
import time

In [ ]:

from leap3d.plotting import plot_top_layer_temperature


fig, ((ax1, ax2, ax3), (ax4, ax5, ax6), (ax7, ax8, ax9)) = plt.subplots(ncols=3, nrows=3, figsize=(20, 20))
ax1.set_aspect('equal')
plot_top_layer_temperature(ax1, rough_temps[:, :, -1], scan_params, vmin=0, vmax=MELTING_POINT)
im = ax1.scatter(laser_x, laser_y, c='red')
ax1.plot([x_points.min(), x_points.min()], [y_points.min(), y_points.max()], 'black', lw=1)
ax1.plot([x_points.max(), x_points.max()], [y_points.min(), y_points.max()], 'black', lw=1)
ax1.plot([x_points.min(), x_points.max()], [y_points.min(), y_points.min()], 'black', lw=1)
ax1.plot([x_points.min(), x_points.max()], [y_points.max(), y_points.max()], 'black', lw=1)
ax1.set_title("Full view")

plot_top_layer_temperature(ax2, rough_temps[:, :, -1], scan_params, vmin=0, vmax=MELTING_POINT)
ax2.set_xlim(x_points.min(), x_points.max())
ax2.set_ylim(y_points.min(), y_points.max())
ax2.set_title("Zoomed-in view (low res)")

melt_pool_coords, melt_pool_temps = scan_results.get_melt_pool_coordinates_and_temperature(timestep)
ax8.set_aspect('equal')
melt_pool_temp = np.array(scan_results.get_melt_pool_temperature_grid(timestep))[:, :, -1]

melt_x_coords = np.arange(x_rough_min, x_rough_max, (x_rough_max - x_rough_min) / 256)
melt_y_coords = np.arange(y_rough_min, y_rough_max, (y_rough_max - y_rough_min) / 256)
print(melt_x_coords.shape, melt_y_coords.shape)
ax3.pcolormesh(melt_x_coords, melt_y_coords, np.transpose(melt_pool_temp), vmin=0, vmax=MELTING_POINT, shading='nearest')
ax3.set_xlim(x_points.min(), x_points.max())
ax3.set_ylim(y_points.min(), y_points.max())
ax3.set_title("Zoomed-in view (high res only)")

methods = ["nearest", "linear", "slinear", "cubic", "quintic", "pchip"]
axes = [ax4, ax5, ax6, ax7, ax8, ax9]

for method, ax in zip(methods, axes):
    t_start = time.time()
    x_points, y_points, z_points, interpolated_temperature = get_interpolated_box_around_laser(rough_coords, scan_results, timestep, 0.25, method=method)
    time_taken = time.time() - t_start
    plot_interolated_top(ax, x_points, y_points, z_points, interpolated_temperature, laser_x, laser_y)
    ax.set_title(f"Interpolated ({method}), {time_taken:.5f}s")

In [ ]:
fig, ax8 = plt.subplots(figsize=(10, 10))
from matplotlib import transforms
melt_pool_coords, melt_pool_temps = scan_results.get_melt_pool_coordinates_and_temperature(timestep)
ax8.set_aspect('equal')
melt_pool_temp = np.array(scan_results.get_melt_pool_temperature_grid(timestep))[:, :, -1]

melt_x_coords = np.arange(x_rough_min, x_rough_max, (x_rough_max - x_rough_min) / 256)
melt_y_coords = np.arange(y_rough_min, y_rough_max, (y_rough_max - y_rough_min) / 256)
print(melt_x_coords.shape, melt_y_coords.shape)
ax8.pcolormesh(melt_x_coords, melt_y_coords, np.transpose(melt_pool_temp), vmin=0, vmax=MELTING_POINT)
ax8.set_xlim(x_points.min(), x_points.max())
ax8.set_ylim(y_points.min(), y_points.max())